<a href="https://colab.research.google.com/github/dnmuturi/SIT742/blob/main/SIT742Task2code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**END TERM ASSESSMENT**

 **GROUP C30 Members**

1. David Muturi - S225177509
2. Nhlanhla Matukane
3. Vincent Nwobi

### **Part 1**

**Answer 1.1**

In [1]:
#install the spark library
!pip install pyspark

In [2]:
#load necessary libraries
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, to_date, when, lit, trim
from google.colab import drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')
# Define the path to the CSV file
csv_path = '/content/drive/My Drive/SIT742/review.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSVCorrectly").getOrCreate()
# Read the CSV file into a PySpark DataFrame with options to improve parsing
try:
    df_review= spark.read.csv(
        csv_path,
        header=True,
        inferSchema=True,
        sep=',',  # Assuming comma is the delimiter
        quote='"', # Assuming double quotes are used for quoting fields
        escape='"', # Assuming double quotes are escaped by double quotes
        multiLine=True # Set to true if text column contains newline characters
    )
    print("CSV file loaded successfully!")
    df_review.show(5, truncate=False) # Display the first five rows without truncating
except Exception as e:
    print(f"An error occurred: {e}")

CSV file loaded successfully!
+---------------------+-----------------+-------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+----+-------------------------------------+
|user_id              |name             |time         |rating|text                                                

In [5]:
#display total no of rows
print(df_review.count())

521515


In [ ]:
# Filter rows where 'text' column is null or an empty string
null_or_empty_text_count = df_review.filter(
    col("text").isNull() | (trim(col("text")) == "")
).count()

# Display the count
print(f"Number of rows with Null or empty text in the text column: {null_or_empty_text_count}")

**Answer 1.1.1**

**Answer 1.1.2**